In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

"""
load and delete social data 

"""

# Import libraries

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
import aerospike
import json
import csv

print(__doc__)

def load(file, ns, sets):
    
    global rec
    rec = {}

    csvfile  = open(file, "rb")
    reader = csv.reader(csvfile)

    config = {
                'hosts': [ ('127.0.0.1', 3000) ]
            }

    # Create a client and connect it to the cluster
    try:
        client = aerospike.client(config).connect()
    except:
        import sys
        print("failed to connect to the cluster with", config['hosts'])
        sys.exit(1)
            
    rownum = 0
    for row in reader:
        # Save First Row with headers
        if rownum == 0:
            header = row
            header = [x.lower() for x in header]
        else:
            colnum = 0
            for col in row:
                # print (rownum,header[colnum],col)
                rec[header[colnum]] = col
                colnum += 1
        rownum += 1
        # print(rownum,rec)
        if rec:
            try:
                client = aerospike.client(config).connect()
                # Write a record
                client.put((ns, sets, str(rownum)), rec)
            except Exception as e:
                import sys
                print("error: {0}".format(e), file=sys.stderr)
        
            # Read a record after insert for verification
            (key, metadata, record) = client.get((ns, sets, str(rownum)))
            print (key, record)
        
            # Close the connection to the Aerospike cluster
            client.close()
        rec = {}

def delete(file, ns, sets):
    
    global rec
    rec = {}

    csvfile  = open(file, "rb")
    reader = csv.reader(csvfile)

    config = {
                'hosts': [ ('127.0.0.1', 3000) ]
            }

    # Create a client and connect it to the cluster
    try:
        client = aerospike.client(config).connect()
    except:
        import sys
        print("failed to connect to the cluster with", config['hosts'])
        sys.exit(1)
            
    rownum = 0
    for row in reader:
        # Save First Row with headers
        if rownum == 0:
            header = row
            header = [x.lower() for x in header]
        else:
            colnum = 0
            for col in row:
                # print (rownum,header[colnum],col)
                rec[header[colnum]] = col
                colnum += 1
        rownum += 1
        # print(rownum,rec)
        if rec:
            try:
                client = aerospike.client(config).connect()
                # Write a record
                client.remove((ns, sets, str(rownum)))
            except Exception as e:
                import sys
                print("error: {0}".format(e), file=sys.stderr)
        
            # Close the connection to the Aerospike cluster
            client.close()
        rec = {}

def drive(mode, file, ns, sets):
    if mode == 'load':
        load(file, ns, sets)
    else:
        delete(file, ns, sets)
        
if __name__ == '__main__':
    #file = '/home/azureuser/Aerospike/data/social/Customer-review-CSV_V1-edited.csv'
    #file = '/home/azureuser/Aerospike/data/social/Dealers_V1-edited.csv'
    #file = '/home/azureuser/Aerospike/data/social/Pricing details_V1.csv'
    file = '/home/azureuser/Aerospike/data/social/Specialoffer_V1.csv'
    ns = 'dev'
    #sets = 'customer_review'
    #sets = 'dealers'
    #sets = 'price'
    sets = 'special_offer'
    drive('load', file, ns, sets)

### Create index for table dev.customer_review

Syntax - CREATE INDEX numindex ON test.testset (binB) NUMERIC|STRING

###### Index 'district_name'
create index i_rev_auth on dev.customer_review (rev_auth) string

### Create index for table dev.dealers

###### Index 'postal_code'
create index i_postal_code on dev.dealers (postal_code) string

### Create index for table dev.price

###### Index 'model'
create index i_model on dev.price (model) string

### Create index for table dev.special_offer

###### Index 'model'
create index i_model on dev.special_offer (model) string